# Basic Usage

HyVR constructs sedimentary facies models from a hierarchy of geobodies assigned to a grid.

In this simple example, we will demonstrate how to create a geobody (object) and how it works. Then we will expand our creativity and create sedimentological virtual realities using these geobodies as building blocks.

### Libraries to run this example
HyVR needs just numpy, numba and scipy. Additionally, to visualize it is desirable to use pyvista for its great functionality in handling 3D meshes and plotting.

In [1]:
import pyvista
import hyvr
import numpy as np

### Grid

HyVR works with structured grids. The simplest way of building a grid is to use numpy meshgrid function.
Here we show an example of a grid creating using the MODFLOW convention axis order (layer, row, col).

In [2]:
xs = np.linspace(0,100,200)
ys = np.linspace(0,80,160)
zs = np.linspace(0,20,50)
xmin = np.min(xs)
xmax = np.max(xs)
ymin = np.min(ys)
ymax = np.max(ys)
z,y,x = np.meshgrid(zs,ys,xs,indexing='ij')
z = np.flip(z,axis=0)
y = np.flip(y,axis=1)

## What is a geobody

From Bennet et al. (2019): "Architectural elements (Miall 1985) are threedimensional sedimentary features (e.g., channels) that are formed by autocyclic processes that occur within depositional systems (Beerbower 1964). They are often superimposed on allocyclic sequences by higher-frequency events that occur over periods of tens to thousands of years (Miall 2013). Different fluvial systems will have their own characteristic architectural elements, and these may range from 101 to 102 m in lateral extent (Miall 1985). Architectural elements are recognized by their outer bounding surfaces, which are often erosional, as well as by their internal facies assemblages (Allen 1983)."

These architectural elements can be simplified as geobodies, which are 3D geometries that represent some architectural element. Currently, HyVR implements 3 different geobodies.

### Trough

The first geobody is the trough. Geometrically it is the lower half of an ellipsoid. It can be used to represent sedimentary architectures such as discontinuous lenses or scour pool fills.

On a basic level, we have to set the center coordinates of the ellipsoid, the dimensions (a,b,c), the azimuth (from North) and the facies index corresponding to that object.

The outputs match the dimension of the grid. they are:

- facies: the facies array with the facies index assigned at the trough location
- dip direction: the angle from east (mathematical convention) of the structure direction (depends on the internal layering)
- dip: the dip angle of the structure (depends on the internal structure settings)


In [3]:
from hyvr import trough

centercoord = np.array([50,40,19])
dims = np.array([40,20,10])
facies, dip, dip_dir = trough(x,y,z,centercoord,
                                                    dims,
                                                    azim = 20.,facies = np.array([1]),
                                                    )

335616
(335616,)
(50, 160, 200)


To visualize with pyvista, the order to the axis is x,y,z which is the transposition of the numpy array according to the `np.tranpose()` function documentation:

In [23]:
x_p = np.transpose(x)
y_p = np.transpose(y)
z_p = np.transpose(z)
facies_p = np.transpose(facies)
print(x_p.shape)

(200, 160, 50)


In [24]:
grid = pyvista.StructuredGrid()
grid.points = np.column_stack((x_p.flatten(), y_p.flatten(), z_p.flatten()))

grid.points = np.column_stack((x_p.flatten(), y_p.flatten(), z_p.flatten())).reshape((200,160,50, 3))

ValueError: Array of points must be 1D or 2D

In [27]:
# Define grid parameters
nx, ny, nz = 10, 10, 10
x = np.linspace(0, 1, nx)
y = np.linspace(0, 1, ny)
z = np.linspace(0, 1, nz)

# Create meshgrid for cell centers
xx, yy, zz = np.meshgrid(x, y, z, indexing='ij')

# Flatten the meshgrid to get the cell centers
points = np.column_stack((xx.flatten(), yy.flatten(), zz.flatten()))

# Create PyVista structured grid
grid = pyvista.StructuredGrid()
grid.points = points

# Reshape the points array to match the dimensions of the grid
grid.points = points.reshape((nx, ny, nz), order='F')

# Save or plot the grid
  # Save the grid to a VTK file
grid.plot(show_scalar_bar=True)   # Plot the grid

ValueError: cannot reshape array of size 3000 into shape (10,10,10)

In [21]:
grid.cell_data["facies"] = facies_p.ravel()
plotter = pyvista.Plotter()
plotter.add_mesh(grid)
plotter.show()

ValueError: data length of (1600000) != required length (0)